In [1]:
import os

In [2]:
%pwd

'c:\\Users\\hp\\Desktop\\ML\\project\\Thyroid_Detection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\hp\\Desktop\\ML\\project\\Thyroid_Detection'

In [65]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/sandeepmande29/Thyroid_Detection.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="sandeepmande29"
os.environ["MLFLOW_TRACKING_PASSWORD"]="2e7eb6f6ace151e1c24e8ab2e48cd8b35037961d"

In [30]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_x_data_path: Path
    test_y_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [31]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [32]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.DecisionTreeClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            
            test_x_data_path=config.test_x_data_path,
            test_y_data_path=config.test_y_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/sandeepmande29/Thyroid_Detection.mlflow",
           
        )

        return model_evaluation_config

In [46]:
import os
import pandas as pd
from sklearn.metrics import precision_score,recall_score,accuracy_score,f1_score
from sklearn import metrics
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [66]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
 
        accuracy=accuracy_score(actual,pred)
        precision=precision_score(actual, pred,average ='micro')
        recall=recall_score(actual,pred,average='micro')
        F1_Score =f1_score(actual,pred,average='micro')
         
        return accuracy,precision,recall,F1_Score
    


    def log_into_mlflow(self):

        test_x = pd.read_csv(self.config.test_x_data_path)
        test_y = pd.read_csv(self.config.test_y_data_path) 
        model = joblib.load(self.config.model_path)

        #test_x = pd.read_csv(self.config.test_x_data_path)
        #test_y = pd.read_csv(self.config.test_y_data_path)

  
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)
            (accuracy,precision,recall,F1_Score) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"accuracy": accuracy, "precision": precision, "recall": recall,"F1_Score":F1_Score}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("F1_Score", F1_Score)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="DecisionTreeClassifierModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [67]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2023-11-08 18:29:26,710: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-08 18:29:26,714: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-08 18:29:26,723: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-11-08 18:29:26,725: INFO: common: created directory at: artifacts]
[2023-11-08 18:29:26,727: INFO: common: created directory at: artifacts/model_evaluation]


[2023-11-08 18:29:27,352: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'DecisionTreeClassifierModel' already exists. Creating a new version of this model...
2023/11/08 18:29:55 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: DecisionTreeClassifierModel, version 4
Created version '4' of model 'DecisionTreeClassifierModel'.
